In [1]:
from langchain.agents.agent_toolkits import PlayWrightBrowserToolkit
from langchain.tools.playwright.utils import (
create_async_playwright_browser,
create_sync_playwright_browser, 
    # A synchronous browser is available, though it isn't compatible with jupyter.
)
from langchain.tools import playwright
import nest_asyncio
import pydantic
import inspect
nest_asyncio.apply()



In [18]:
# help(playwright.get_elements.GetElementsToolInput)
inspect.getsourcefile(playwright)

'/home/ubuntu/.local/lib/python3.11/site-packages/langchain/tools/playwright/__init__.py'

In [3]:
async_browser = create_async_playwright_browser()
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()
tools

[ClickTool(name='click_element', description='Click on an element with the given CSS selector', args_schema=<class 'langchain.tools.playwright.click.ClickToolInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False, sync_browser=None, async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/ubuntu/.cache/ms-playwright/chromium-1067/chrome-linux/chrome> version=115.0.5790.24>, visible_only=True, playwright_strict=False, playwright_timeout=1000),
 NavigateTool(name='navigate_browser', description='Navigate a browser to the specified URL', args_schema=<class 'langchain.tools.playwright.navigate.NavigateToolInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False, sync_browser=None, async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/ubuntu/.cache/ms-playwright/chromium-1067/chrome-linux/chrome> version=115.0.5790.24>),
 NavigateBackTool(name='p

In [5]:
tools_by_name = {tool.name: tool for tool in tools}
navigate_tool = tools_by_name["navigate_browser"]
get_elements_tool = tools_by_name["get_elements"]

await navigate_tool.arun(
    {"url": "https://web.archive.org/web/20230428131116/https://www.cnn.com/world"}
)

TimeoutError: Timeout 30000ms exceeded.
=========================== logs ===========================
navigating to "https://web.archive.org/web/20230428131116/https://www.cnn.com/world", waiting until "load"
============================================================

In [6]:
await get_elements_tool.arun(
    {"selector": ".container__headline", "attributes": ["innerText"]}
)

'[{"innerText": "These Ukrainian veterinarians are risking their lives to care for dogs and cats in the war zone"}, {"innerText": "Life in the ocean’s ‘twilight zone’ could disappear due to the climate crisis"}, {"innerText": "Clashes renew in West Darfur as food and water shortages worsen in Sudan violence"}, {"innerText": "Thai policeman’s wife investigated over alleged murder and a dozen other poison cases"}, {"innerText": "American teacher escaped Sudan on French evacuation plane, with no help offered back home"}, {"innerText": "Dubai’s emerging hip-hop scene is finding its voice"}, {"innerText": "How an underwater film inspired a marine protected area off Kenya’s coast"}, {"innerText": "The Iranian drones deployed by Russia in Ukraine are powered by stolen Western technology, research reveals"}, {"innerText": "India says border violations erode ‘entire basis’ of ties with China"}, {"innerText": "Australian police sift through 3,000 tons of trash for missing woman’s remains"}, {"in

In [7]:
await tools_by_name["current_webpage"].arun({})

'https://web.archive.org/web/20230428133211/https://cnn.com/world'

In [11]:
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
import dotenv

dotenv.load_dotenv()

llm = ChatOpenAI(temperature=0)  # or any other LLM, e.g., ChatOpenAI(), OpenAI()

agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

result = await agent_chain.arun("What are the headers on https://shub.edu.vn/?")
print(result)



> Entering new  chain...
I can use the `navigate_browser` and `extract_text` tools to get the headers on the website.

Action:
```
{
  "action": "navigate_browser",
  "action_input": {
    "url": "https://shub.edu.vn/"
  }
}
```



TimeoutError: Timeout 30000ms exceeded.
=========================== logs ===========================
navigating to "https://shub.edu.vn/", waiting until "load"
============================================================